## We have developed a demonstration of the LBN-MOBO algorithm applied to the ZDT3 problem (30 dimensions). The code is designed to be easily adaptable to different datasets.

Create a new directory with the name **LBN-MOBO** and transfer the project in that folder. Connect the colab to your google drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Change the directory to the project directory.

In [2]:
import os
os.chdir('/content/drive/MyDrive/LBN-MOBO')
cwd = os.getcwd()
print(cwd)

/content/drive/MyDrive/LBN-MOBO


Install the missing libraries. 

In [3]:
!pip install -U pymoo
!pip install pygmo
!pip install Platypus-Opt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Run **LBN-MOBO**. **Note** that here the deep ensemble models are trained sequentially while in reality they are run in parallel on multiple GPUs.

In [ ]:
from Random_dataset_gen import random_dset_gen
from BO_surrogate_function_uncertainty import BO_surrogate_uncertainty
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
import matplotlib.pyplot as plt
from query_oracle import Oracle_function
import scipy.io as sio
import numpy as np
import os
import subprocess
import time
from acquisition_organizeer import dset_organizer
from Forward_BNN import BNN_diverse_func
from Acquisition_4_D import calculate_pareto_4_D

# Each iteration of NSGA-II is set to generate 1000 samples as a result choose a multiplication of 1000 as batch size (2000, 3000, ...).
Batch_size = 2000 
problem_name = "zdt3"
# Generate the initial random data
dset = random_dset_gen(problem_name, Batch_size)
sio.savemat('Dataset/dset_0.mat', {'dset': dset})

data_folder = 'surrogate_pareto/with_uncertainty/'
elapsed_times = [] 
for n_iter in range(10):
    start_time = time.time()
    # # Train the BNN
    #make the path for the models
    new_dir = 'Models/iter_%d' %(n_iter)
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)
        print("Directory created:", new_dir)
    else:
        print("Directory already exists:", new_dir)
    # BNN model training
    print('Training the models ...')
    for n_net in range(10):
      BNN_diverse_func(n_net, n_iter)
    dir_path = 'Models/iter_%d' %(n_iter)
    while True:
        # Get the number of files in the directory
        num_files = (len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))

        # If there are 10 files, break out of the loop
        if num_files == 10:
            break
        # If there are less than 10 files, wait for 1 second and try again
        time.sleep(1)


    # Acquisition
    print('Running acquisition ...')
        # make the path for the acquisition
    new_dir = 'Acquisition/iter_%d' % (n_iter)
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)
        print("Directory created:", new_dir)
    else:
        print("Directory already exists:", new_dir)
    # Run acquisitions n=(Batch_size/1000) times. Each time it generates different 1000 samples
    for NSGA2_seed in range(int(Batch_size/1000)):
      calculate_pareto_4_D(n_iter, NSGA2_seed)
    dir_path_acquisition = 'Acquisition/iter_%d' %(n_iter)
    while True:
        # Get the number of files in the directory
        num_files = (len([entry for entry in os.listdir(dir_path_acquisition) if os.path.isfile(os.path.join(dir_path_acquisition, entry))]))
        # If there are n=(Batch_size/1000) files, break out of the loop
        if num_files ==  int(Batch_size/1000):
            break
        # If there are less than n=(Batch_size/1000) files, wait for 1 second and try again
        time.sleep(1)

    dset_new = dset_organizer(n_iter, int(Batch_size/1000))

    # Stack the newly generated candidates into the dataset
    mat = sio.loadmat('Dataset/dset_%d.mat' %(n_iter))
    mat = mat['dset']
    dset = mat[0][0]


    dset['X'] = np.concatenate((dset['X'], dset_new['X']))
    dset['Y'] = np.concatenate((dset['Y'], dset_new['Y']))
    end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_times.append(elapsed_time)
    sio.savemat('Dataset/dset_%d.mat' %(n_iter+1), {'dset': dset})

elapsed_times_arr = np.array(elapsed_times)
output_filename = "execution_times.mat"  # Output .mat file name
sio.savemat(output_filename, {'elapsed_times': elapsed_times_arr})


Directory already exists: Models/iter_0
Training the models ...
0
Using device: cpu
13452
Epoch [1/1], Step [100/180], Loss: 2.9557
Validataion MSE is: 0.3421275019645691
1
Using device: cpu
13452
Epoch [1/1], Step [100/180], Loss: 2.1111
Validataion MSE is: 0.2891792953014374
2
Using device: cpu
13452
Epoch [1/1], Step [100/180], Loss: 1.3727
Validataion MSE is: 0.30941903591156006
3
Using device: cpu
13452
Epoch [1/1], Step [100/180], Loss: 2.9626
Validataion MSE is: 0.30325639247894287
4
Using device: cpu
13452
Epoch [1/1], Step [100/180], Loss: 2.4432
Validataion MSE is: 0.28618675470352173
5
Using device: cpu
13452
Epoch [1/1], Step [100/180], Loss: 2.9243
Validataion MSE is: 0.2892408072948456
6
Using device: cpu
13452
Epoch [1/1], Step [100/180], Loss: 5.6079
Validataion MSE is: 0.29327067732810974
7
Using device: cpu
13452
Epoch [1/1], Step [100/180], Loss: 1.7221
Validataion MSE is: 0.2725880444049835
8
Using device: cpu
13452
Epoch [1/1], Step [100/180], Loss: 4.4264
Validata

Plot the hypervolume progress of different optimization algorithm. Note that since USeMO, TSEMO, and DGEMO algorithms are time-consuming to calculate we provine the pre-computed data for the sake of comparison (Figure 3 in the paper). The slight difference comes from the different algorithms that we have used to calculate the hypervolume in python.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload 
import hv_calculator
hv_calculator_ = reload(hv_calculator)


hyper_val_LBN_MOBO, ParetoFront_LBN_MOBO = hv_calculator_.hv_ploter(10, './surrogate_pareto/ParetoFront_%d.mat', 0)
hyper_val_DGEMO, _ = hv_calculator_.hv_ploter(7, './DGEMO_data/DGEMO_pareto_iter_%d.mat', 1)
hyper_val_USeMO, _  = hv_calculator_.hv_ploter(7, './USeMO_data/USEMO_pareto_iter_%d.mat', 1)
hyper_val_TSEMO, _  = hv_calculator_.hv_ploter(7, './TSEMO_data/TSEMO_pareto_iter_%d.mat', 1)
hyper_val_NSGA2, _  = hv_calculator_.hv_ploter(10, './NSGA2_data/NSGA2_pareto_iter_%d.mat', 1)



# Plot the hyper volume of LBN-MOBO
plt.step(np.arange(1,11), hyper_val_LBN_MOBO)
plt.step(np.arange(1,8), hyper_val_DGEMO)
plt.step(np.arange(1,8), hyper_val_USeMO)
plt.step(np.arange(1,8), hyper_val_TSEMO)
plt.step(np.arange(1,11), hyper_val_NSGA2)

# Customize the plot
plt.xlabel('Number of iterations')
plt.ylabel('Hypervolume')
plt.title('ZDT 3 Problem')
plt.legend(['LBN-MOBO', 'DGEMO', 'USeMO', 'TSEMO', 'NSGA2'])

# Show the plot
plt.show()

Plot the final paretofront after 10 iterations.

In [ ]:
import scipy.io as sio

def plot_pareto_frontier(Xs, Ys, maxX=True, maxY=True):
    '''Pareto frontier selection process'''
    sorted_list = sorted([[Xs[i], Ys[i]] for i in range(len(Xs))], reverse=maxY)
    pareto_front = [sorted_list[0]]
    for pair in sorted_list[1:]:
        if maxY:
            if pair[1] >= pareto_front[-1][1]:
                pareto_front.append(pair)
        else:
            if pair[1] <= pareto_front[-1][1]:
                pareto_front.append(pair)
    
    # Load and plot the GT
    mat = sio.loadmat('GT_data.mat')
    ParetoFront = mat['GT_data']
    plt.scatter(ParetoFront[:,0], ParetoFront[:,1])
    # plt.scatter(-Xs, -Ys, c='red')
    pf_X = [-pair[0] for pair in pareto_front]
    pf_Y = [-pair[1] for pair in pareto_front]
    plt.scatter(pf_X, pf_Y, marker='*')
    plt.xlabel("Objective 1")
    plt.ylabel("Objective 2")
    plt.legend(['GT', 'LBN-MOBO'])

    
    plt.xlim(0, 1)
    plt.ylim(-1, 1)
    plt.show()


plot_pareto_frontier(-ParetoFront_LBN_MOBO[:,0], -ParetoFront_LBN_MOBO[:,1], maxX=True, maxY=True)